Sisi Koding
1. Rapihin preprocessingnya
2. Jalankan HDP untuk liat top 30 nya (baru setelah ini bisa kabarin aku dulu biar aku liat list topiknya dan aku bisa tentuin aspeknya (kelarin HDP) 
3. Topik yang aku pilih akan menjadi tolak ukur untuk Analisa sentimennya. Sekaligus merapihan data-data yang akan di mapping ke aspek tersebut 
mau, baik, bahan, warna, kain.
warna ukuran bahan = 

4. Analisa sentiment berbasis aspeknya menggunakan SVM dan Indobert -> hasil akhir mirip 

interface :
svm & indobert -> harga berapa positif & negatif, bahan berapa positif & negatif, warna berapa positif & negatif

program bisa diakses tanpa coding

1 menu : bisa masukin link

- crawling data (sentimen negatif 50 positif 50 (rate1-3 50%, rate 4-5 50%))
- segmentasi kain (lembut, halus, dll) -> (3-5 aspek : warna, bahan, kualitas)
- barchart kosongin jika gada sesuai dengan aspek (hasil ulasan tidak mengandung tema bahan, warna, dan kain!)

###  catatan :
1. segment -> aspek
2. bar -> merge (merah -> negatif, biru -> positif) -> bar nempel
3. mencari coherence terbaik yang memuat 3 segement/aspek
4. ### HDP (reference codingan & perhitungan) -> screenshoot hasil hdp
5. konfigurasi path : HDP, interface (dashboard)

reference HDP : 
1. https://github.com/bab2min/tomotopy
2. https://mlg.eng.cam.ac.uk/zoubin/tut06/ywt.pdf
3. https://link.springer.com/article/10.1007/s10994-016-5621-5

In [7]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import HdpModel
from gensim.models.coherencemodel import CoherenceModel

In [20]:
# Unduh stopwords dan wordnet jika belum diunduh
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Baca data ulasan dari file CSV
df = pd.read_csv("data/dataHasilPreprocessing/dataPreprocessing.csv")

# Mengisi nilai NaN dengan string kosong
df['Ulasan'] = df['Ulasan'].fillna('')

# Inisialisasi stop words dan lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Fungsi preprocessing
def preprocess(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Hapus stop words dan tanda baca, lakukan lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Preprocessing data ulasan
df['processed'] = df['Ulasan'].apply(preprocess)

# Buat dictionary dan corpus
dictionary = corpora.Dictionary(df['processed'])
corpus = [dictionary.doc2bow(text) for text in df['processed']]

# Latih model HDP
hdp = HdpModel(corpus, dictionary)

# Ekstrak topik dan kata kunci
top_topics = hdp.show_topics(num_topics=10, num_words=10, formatted=False)
top_words = []

# Daftar kata kunci untuk setiap segmentasi
segmentation_keywords = {
    'bahan': ['tipis', 'tebal', 'lembut', 'keras', 'kasar', 'rapih', 'rapi', 'pendek', 'adem', 'nyaman', 'jahit', 'halus', 'gerah', 'relaxing', 'baju', 'model', 'celana', 'nama', 'transparan', 'badan', 'sayap'],
    'pengiriman': ['cepat', 'lambat', 'lelet', 'ontime', 'terlambat', 'instan', 'kurir', 'ekspektasi', 'semangat', 'kilogram', 'packingan', 'super', 'gampang', 'kilo', 'ekspedisi', 'online', 'kirim', 'diiket', 'angkat', 'realita'],
    'kualitas': ['rusak', 'sesuai', 'bagus', 'jelek', 'berkualitas', 'keringat', 'sobek', 'aneh', 'foto', 'gambar', 'keren', 'mantap', 'kecil', 'label', 'ngetat', 'ketat', 'pict', 'fashion', 'bolong', 'style', 'sederhana'],
    'warna': ['cerah', 'pudar', 'gelap', 'putih', 'hitam', 'warna', 'biru', 'soft', 'navy', 'pink'],
    'harga': ['murah', 'mahal', 'terjangkau', 'ekonomis', 'premium', 'uang', 'duit', 'refund', 'promo', 'promonya'],
    'respon': ['ragu', 'tidak puas', 'kurang', 'positif', 'negatif', 'astaga', 'tengkyuu', 'emang', 'tanggap', 'nyoba', 'suka', 'worth', 'haha', 'tolong', 'banget', 'balas', 'thanks', 'thank', 'aduh']
}

# Fungsi untuk menentukan segmentasi
def categorize_word(word):
    for segment, keywords in segmentation_keywords.items():
        if word in keywords:
            return segment
    return 'other'

# Mengelompokkan kata berdasarkan segmentasi
segmentation_words = []

for i, topic in top_topics:
    for word, weight in topic:
        segment = categorize_word(word)
        segmentation_words.append({'Segmentasi': segment, 'Kata': word, 'Bobot': weight})

# Buat DataFrame untuk kata
# a-kata berdasarkan segmentasi
segmentation_words_df = pd.DataFrame(segmentation_words)
segmentation_summary_df = segmentation_words_df.groupby(['Segmentasi', 'Kata']).agg({'Bobot': 'max'}).reset_index()
segmentation_summary_df = segmentation_summary_df.sort_values(by=['Segmentasi', 'Bobot'], ascending=[True, False])

print("\nKata-Kata Berdasarkan Segmentasi:")
segmentation_summary_df

[nltk_data] Downloading package stopwords to C:\Users\Muhammad Ade
[nltk_data]     Aulia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Muhammad Ade
[nltk_data]     Aulia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Muhammad Ade
[nltk_data]     Aulia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Kata-Kata Berdasarkan Segmentasi:


,Segmentasi,Kata,Bobot
0,bahan,kasar,0.005036
1,harga,murah,0.004019
5,kualitas,style,0.005482
3,kualitas,kecil,0.004091
4,kualitas,sederhana,0.004029
...,...,...,...
93,pengiriman,kurir,0.006247
94,pengiriman,packingan,0.005161
95,respon,banget,0.004220
96,warna,pink,0.005438


In [9]:
# Hitung nilai coherence
coherence_model = CoherenceModel(model=hdp, texts=df['processed'].tolist(), dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

print(f"\nNilai Coherence: {coherence_score}")


Nilai Coherence: 0.7851410845906054
